In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [35]:
# Lee la base de datos
df = pd.read_csv("../data/processed/grd_hmd.csv")
df["fech_alta"] = pd.to_datetime(df["fech_alta"])

# Separa en actividad hospitalaria y Hemodinamia
df_egresos = df.query("tipo_actividad_tipo == 'H'").copy()
df_hmd = df.query("tipo_actividad_tipo == 'A'").copy()

C:\Users\javie\AppData\Local\Temp\ipykernel_31568\1780020483.py:2: DtypeWarning: Columns (87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,161) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/grd_hmd.csv")
